In [1]:
import gym
import random
import numpy as np
from keras.models     import Sequential
from keras.layers     import Dense
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
env = gym.make('CartPole-v1')
env.reset()
goal_steps = 500
score_requirement = 60
intial_games = 10000

In [3]:
def play_a_random_game_first():
    for step_index in range(goal_steps):
#         env.render()
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        print("Step {}:".format(step_index))
        print("action: {}".format(action))
        print("observation: {}".format(observation))
        print("reward: {}".format(reward))
        print("done: {}".format(done))
        print("info: {}".format(info))
        if done:
            break
    env.reset()

In [4]:
play_a_random_game_first()

Step 0:
action: 1
observation: [-0.02540119  0.2303322  -0.04179892 -0.29672463]
reward: 1.0
done: False
info: {}
Step 1:
action: 1
observation: [-0.02079454  0.42602432 -0.04773341 -0.60229172]
reward: 1.0
done: False
info: {}
Step 2:
action: 1
observation: [-0.01227406  0.6217803  -0.05977925 -0.90961953]
reward: 1.0
done: False
info: {}
Step 3:
action: 0
observation: [ 1.61548575e-04  4.27516139e-01 -7.79716371e-02 -6.36308427e-01]
reward: 1.0
done: False
info: {}
Step 4:
action: 0
observation: [ 0.00871187  0.23356322 -0.09069781 -0.36916341]
reward: 1.0
done: False
info: {}
Step 5:
action: 0
observation: [ 0.01338314  0.03983913 -0.09808107 -0.10640026]
reward: 1.0
done: False
info: {}
Step 6:
action: 0
observation: [ 0.01417992 -0.15375045 -0.10020908  0.15379887]
reward: 1.0
done: False
info: {}
Step 7:
action: 1
observation: [ 0.01110491  0.04265291 -0.0971331  -0.1687404 ]
reward: 1.0
done: False
info: {}
Step 8:
action: 0
observation: [ 0.01195797 -0.15095421 -0.10050791  0.0

In [5]:
def model_data_preparation():
    training_data = []
    accepted_scores = []
    for game_index in range(intial_games):
        score = 0
        game_memory = []
        previous_observation = []
        for step_index in range(goal_steps):
            action = random.randrange(0, 2)
            observation, reward, done, info = env.step(action)
            
            if len(previous_observation) > 0:
                game_memory.append([previous_observation, action])
                
            previous_observation = observation
            score += reward
            if done:
                break
            
        if score >= score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                if data[1] == 1:
                    output = [0, 1]
                elif data[1] == 0:
                    output = [1, 0]
                training_data.append([data[0], output])
        
        env.reset()

    print(accepted_scores)
    
    return training_data

In [6]:
training_data = model_data_preparation()

[63.0, 68.0, 88.0, 63.0, 60.0, 65.0, 72.0, 62.0, 60.0, 60.0, 70.0, 60.0, 90.0, 63.0, 89.0, 83.0, 67.0, 62.0, 67.0, 68.0, 63.0, 98.0, 67.0, 113.0, 75.0, 61.0, 69.0, 85.0, 89.0, 70.0, 78.0, 60.0, 72.0, 67.0, 82.0, 81.0, 79.0, 67.0, 68.0, 66.0, 60.0, 61.0, 73.0, 70.0, 62.0, 61.0, 88.0, 89.0, 85.0, 92.0, 63.0, 60.0, 61.0, 65.0, 67.0, 69.0, 69.0, 68.0, 61.0, 67.0, 84.0, 60.0, 62.0, 69.0, 70.0, 63.0, 81.0, 68.0, 63.0, 63.0, 101.0, 61.0, 69.0, 68.0, 77.0, 63.0, 74.0, 63.0, 63.0, 67.0, 67.0, 84.0, 60.0, 93.0, 91.0, 81.0, 62.0, 65.0, 71.0, 85.0, 62.0, 63.0, 66.0, 65.0, 86.0, 61.0, 65.0, 83.0, 67.0, 73.0, 62.0, 64.0, 80.0, 61.0, 60.0, 78.0, 63.0, 64.0, 65.0, 68.0, 88.0, 74.0, 75.0, 63.0, 68.0, 68.0, 74.0, 69.0, 64.0, 71.0, 134.0, 61.0, 93.0, 72.0, 62.0, 61.0, 68.0, 68.0, 63.0, 66.0, 70.0, 82.0, 65.0, 90.0, 90.0, 80.0, 61.0, 62.0, 70.0, 64.0, 79.0, 91.0, 76.0, 69.0, 61.0, 76.0, 70.0, 61.0, 71.0, 67.0, 76.0, 93.0, 81.0, 60.0, 73.0, 61.0, 70.0, 80.0, 67.0, 92.0, 76.0, 75.0, 90.0, 65.0, 67.0, 79.0, 

In [7]:
def build_model(input_size, output_size):
    model = Sequential()
    model.add(Dense(128, input_dim=input_size, activation='relu'))
    model.add(Dense(52, activation='relu'))
    model.add(Dense(output_size, activation='linear'))
    model.compile(loss='mse', optimizer=Adam())

    return model

In [8]:
def train_model(training_data):
    X = np.array([i[0] for i in training_data]).reshape(-1, len(training_data[0][0]))
    y = np.array([i[1] for i in training_data]).reshape(-1, len(training_data[0][1]))
    model = build_model(input_size=len(X[0]), output_size=len(y[0]))
    
    model.fit(X, y, epochs=10)
    return model

In [9]:
trained_model = train_model(training_data)

Epoch 1/10
11956/11956 [==============================] - 2s 193us/step - loss: 0.2442
Epoch 2/10
11956/11956 [==============================] - 2s 179us/step - loss: 0.2348
Epoch 3/10
11956/11956 [==============================] - 3s 214us/step - loss: 0.2339
Epoch 4/10
11956/11956 [==============================] - 3s 213us/step - loss: 0.2331
Epoch 5/10
11956/11956 [==============================] - 2s 174us/step - loss: 0.2327
Epoch 6/10
11956/11956 [==============================] - 2s 206us/step - loss: 0.2324
Epoch 7/10
11956/11956 [==============================] - 2s 163us/step - loss: 0.2323
Epoch 8/10
11956/11956 [==============================] - 2s 204us/step - loss: 0.2317
Epoch 9/10
11956/11956 [==============================] - 2s 206us/step - loss: 0.2316
Epoch 10/10
11956/11956 [==============================] - 3s 228us/step - loss: 0.2316


In [11]:
scores = []
choices = []
for each_game in range(500):
    score = 0
    prev_obs = []
    for step_index in range(goal_steps):
        # Uncomment below line if you want to see how our bot is playing the game.
        # env.render()
        if len(prev_obs)==0:
            action = random.randrange(0,2)
        else:
            action = np.argmax(trained_model.predict(prev_obs.reshape(-1, len(prev_obs)))[0])
        
        choices.append(action)
        new_observation, reward, done, info = env.step(action)
        prev_obs = new_observation
        score+=reward
        if done:
            break
    env.render()
    env.reset()
    scores.append(score)

print(scores)
print('Average Score:',sum(scores)/len(scores))
print('choice 1:{}  choice 0:{}'.format(choices.count(1)/len(choices),choices.count(0)/len(choices)))

[500.0, 252.0, 211.0, 220.0, 261.0, 185.0, 158.0, 323.0, 197.0, 500.0, 329.0, 500.0, 213.0, 500.0, 312.0, 243.0, 157.0, 368.0, 279.0, 85.0, 148.0, 137.0, 321.0, 91.0, 263.0, 159.0, 127.0, 200.0, 500.0, 147.0, 324.0, 193.0, 263.0, 390.0, 298.0, 177.0, 500.0, 183.0, 183.0, 107.0, 113.0, 309.0, 500.0, 500.0, 164.0, 301.0, 158.0, 97.0, 500.0, 500.0, 326.0, 83.0, 205.0, 500.0, 95.0, 257.0, 500.0, 83.0, 142.0, 219.0, 125.0, 148.0, 115.0, 500.0, 500.0, 79.0, 136.0, 249.0, 95.0, 217.0, 262.0, 74.0, 74.0, 133.0, 154.0, 248.0, 500.0, 500.0, 209.0, 96.0, 500.0, 316.0, 75.0, 107.0, 229.0, 194.0, 103.0, 219.0, 83.0, 500.0, 137.0, 282.0, 252.0, 500.0, 500.0, 143.0, 170.0, 182.0, 500.0, 206.0]
Average Score: 256.68
choice 1:0.49594826242792583  choice 0:0.5040517375720742
